# General Setup

Install CR-VISION Package

In [1]:
!python -m pip install --quiet git+https://github.com/carnotresearch/cr-vision.git


     |████████████████████████████████| 2.3MB 19.2MB/s 
     |████████████████████████████████| 204kB 54.7MB/s 
     |████████████████████████████████| 51kB 8.5MB/s 
     |████████████████████████████████| 9.2MB 50.2MB/s 
     |████████████████████████████████| 563kB 57.1MB/s 
     |████████████████████████████████| 2.8MB 49.2MB/s 
     |████████████████████████████████| 122kB 58.9MB/s 
     |████████████████████████████████| 102kB 14.2MB/s 
     |████████████████████████████████| 92kB 13.0MB/s 
     |████████████████████████████████| 92kB 13.4MB/s 
ERROR: sphinx 3.5.4 has requirement docutils<0.17,>=0.12, but you'll have docutils 0.17 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: sphinx-rtd-theme 0.5.2 has requirement docutils<0.17, but you'll have docutils 0.17 which is incompatible.


Mount Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Copy  and Unzip Training and Validation Images

In [3]:
!cp  /content/drive/MyDrive/datasets/birds/birds_subset_5000/training.zip .

In [4]:
!unzip -q training.zip

In [5]:
!cp  /content/drive/MyDrive/datasets/birds/birds_subset_5000/validation.zip .

In [6]:
!unzip -q validation.zip

Copy  driver code for the experiment setup

In [24]:
!cp /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/csresnet.py .

Import essential packages



In [8]:
from pathlib import Path
import pandas as pd

Import utilities from CR-Vision package

In [9]:

from cr.vision.io import images_from_dir


Import driver module for the experiment



In [10]:
import csresnet

# Auto Encoder Model Preparation

Specify model definition parameters

In [11]:
patch_size = 24
stride_size = 16
compression_ratio = 64
num_res_blocks=2
num_res_channels=32
input_shape  = (256, 256, 3)

Build autoencoder model

In [12]:
models = csresnet.build_models(input_shape,
    patch_size=patch_size,
    stride_size=stride_size,
    num_res_blocks=num_res_blocks,
    num_res_channels=num_res_channels,
    compression_ratio=compression_ratio)

In [13]:
autoencoder = models.autoencoder
autoencoder.summary()

Model: "CSResNet_Autoencoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256, 256, 3)]     0         
_________________________________________________________________
CSResNet_Encoder (Functional (None, 16, 16, 12)        20748     
_________________________________________________________________
CSResNet_Decoder (Functional (None, 256, 256, 3)       240579    
Total params: 261,327
Trainable params: 261,327
Non-trainable params: 0
_________________________________________________________________


In [14]:
encoder = models.encoder
encoder.summary()

Model: "CSResNet_Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 256, 256, 3)]     0         
_________________________________________________________________
sensor (Conv2D)              (None, 16, 16, 12)        20748     
Total params: 20,748
Trainable params: 20,748
Non-trainable params: 0
_________________________________________________________________


In [15]:
decoder = models.decoder
decoder.summary()

Model: "CSResNet_Decoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoded_input (InputLayer)      [(None, 16, 16, 12)] 0                                            
__________________________________________________________________________________________________
initial (Conv2DTranspose)       (None, 256, 256, 32) 221216      encoded_input[0][0]              
__________________________________________________________________________________________________
residual-1 (Conv2D)             (None, 256, 256, 32) 9248        initial[0][0]                    
__________________________________________________________________________________________________
add-1 (Add)                     (None, 256, 256, 32) 0           initial[0][0]                    
                                                                 residual-1[0][0]  

Compile model

In [16]:
csresnet.compile_model(autoencoder)

Compiling model for training.


Add callbacks

In [17]:
callbacks = csresnet.build_callbacks()


Preparing callbacks for model training.


# Training and validation data

Load training and validation images

In [18]:
rootdir = Path('.')
training_set = images_from_dir(rootdir / 'training', size=1200)
validation_set = images_from_dir(rootdir / 'validation', size=400)


training/025.Pelagic_Cormorant_Pelagic_Cormorant_0022_23802.jpg, (375, 500)
shape after conversion (375, 500, 3)
training/063.Ivory_Gull_Ivory_Gull_0040_49180.jpg, (326, 500)
shape after conversion (326, 500, 3)


In [19]:
print(f"training set: {training_set.shape}")
print(f"validation set: {validation_set.shape}")

training set: (1200, 256, 256, 3)
validation set: (400, 256, 256, 3)


# Training setup

Specify training paramters

In [20]:
batch_size = 32
epochs=50
n_train = training_set.shape[0]
steps_per_epoch = n_train // batch_size

Prepare training data augmentation generator

In [21]:
train_gen = csresnet.augment_training_set(training_set, batch_size = batch_size)

Preparing augmented training set generator.


# Training

In [22]:
history = csresnet.fit_model(autoencoder, train_gen,
    validation_set, callbacks,
    steps_per_epoch=steps_per_epoch,
    epochs=epochs)

Initiating model training for 50 epochs.
37 50
(400, 256, 256, 3)
Epoch 1/50
37/37 [==============================] - 64s 836ms/step - loss: 0.0663 - val_loss: 0.0190

Epoch 00001: val_loss improved from inf to 0.01900, saving model to /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/checkpoint.hdf5
Epoch 2/50
37/37 [==============================] - 30s 810ms/step - loss: 0.0166 - val_loss: 0.0120

Epoch 00002: val_loss improved from 0.01900 to 0.01198, saving model to /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/checkpoint.hdf5
Epoch 3/50
37/37 [==============================] - 29s 802ms/step - loss: 0.0106 - val_loss: 0.0093

Epoch 00003: val_loss improved from 0.01198 to 0.00934, saving model to /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/checkpoint.hdf5
Epoch 4/50
37/37 [==============================] - 29s 802ms/step - loss: 0.0085 - val_loss: 0.0083

Epoch 00004: val_loss improved from 0.00934 to 0.00831, saving model to /content

# Saving of trained model and training history

In [28]:
csresnet.save_model(models.autoencoder, csresnet.form_model_name('autoencoder', patch_size, stride_size, num_res_blocks, num_res_channels, compression_ratio))
csresnet.save_model(models.encoder, csresnet.form_model_name('encoder', patch_size, stride_size, num_res_blocks, num_res_channels, compression_ratio))
csresnet.save_model(models.decoder, csresnet.form_model_name('decoder', patch_size, stride_size, num_res_blocks, num_res_channels, compression_ratio))

Saving autoencoder_p=24_s=16_b=2_c=32_cr=64 in tensorflow SavedModel format.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/saved_model_autoencoder_p=24_s=16_b=2_c=32_cr=64/assets
Saving encoder_p=24_s=16_b=2_c=32_cr=64 in tensorflow SavedModel format.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/saved_model_encoder_p=24_s=16_b=2_c=32_cr=64/assets
Saving decoder_p=24_s=16_b=2_c=32_cr=64 in tensorflow SavedModel format.
INFO:tensorflow:Assets written to: /content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet/saved_model_decoder_p=24_s=16_b=2_c=32_cr=64/assets


In [29]:
name = csresnet.form_model_name('history', patch_size, stride_size, num_res_blocks, num_res_channels, compression_ratio)
history_file = f'{csresnet.GD_EXP_DIR}_{name}.json'
history_file

'/content/drive/MyDrive/work/cr-vision/experiments/cs/csresnet_history_p=24_s=16_b=2_c=32_cr=64.json'

In [30]:
hist_df = pd.DataFrame(history.history)

In [31]:
with open(history_file, mode='w') as f:
        hist_df.to_json(f)